In [ ]:
!pip install tensorflow
import os

# Defining each of these directories

In [ ]:
#base_dir = '../input/mechanical-tools-dataset/Mechanical Tools Image dataset'

train_dir = os.path.join('../input/mechanical-tools-dataset/train_data_V2/train_data_V2')
validation_dir = os.path.join('../input/mechanical-tools-dataset/validation_data_V2/validation_data_V2')

# Directory with our training screwdriver/wrench pictures
train_screwdriver_dir = os.path.join('../input/mechanical-tools-dataset/train_data_V2/train_data_V2/screwdriver')
train_wrench_dir = os.path.join('../input/mechanical-tools-dataset/train_data_V2/train_data_V2/wrench')
train_hammer_dir = os.path.join('../input/mechanical-tools-dataset/train_data_V2/train_data_V2/hammer')

# Directory with our validation screwdriver/wrench pictures
validation_screwdriver_dir = os.path.join('../input/mechanical-tools-dataset/validation_data_V2/validation_data_V2/screwdriver')
validation_wrench_dir = os.path.join('../input/mechanical-tools-dataset/validation_data_V2/validation_data_V2/wrench')
validation_hammer_dir = os.path.join('../input/mechanical-tools-dataset/validation_data_V2/validation_data_V2/hammer')

**Now, let's see what the filenames look like in the training directories:**

In [ ]:
train_screwdriver_fnames = os.listdir(train_screwdriver_dir )
train_wrench_fnames = os.listdir( train_wrench_dir)
train_hammer_fnames = os.listdir( train_hammer_dir )

print(train_screwdriver_fnames[:20])
print(train_wrench_fnames[:20])
print(train_hammer_fnames[:20])

**Let's find out the number of wrench and pliers images in the directory**

In [ ]:
print('total training screwdriver images :', len(os.listdir(train_screwdriver_dir)))
print('total training wrench images :', len(os.listdir(train_wrench_dir)))
print('total training hammer images :', len(os.listdir(train_hammer_dir)))


print('total validation screwdriver images :', len(os.listdir( validation_screwdriver_dir ) ))
print('total validation wrench images :', len(os.listdir( validation_wrench_dir) ))
print('total validation hammer images :', len(os.listdir( validation_hammer_dir) ))

**Now let's take a look at a few pictures to get a better sense of what they look like. First, configure the matplot parameters:**

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 10x10 configuration
nrows = 10
ncols = 10

# Index for iterating over images
pic_index = 0

**Now, display a batch of 50 wrench and 50 pliers pictures. You can rerun the cell to see a fresh batch each time:**

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 10
next_screwdriver_pix = [os.path.join(train_screwdriver_dir, fname) 
                for fname in train_screwdriver_fnames[pic_index-10:pic_index]]
next_wrench_pix = [os.path.join(train_wrench_dir, fname) 
                for fname in train_wrench_fnames[pic_index-10:pic_index]]
next_hammer_pix = [os.path.join(train_hammer_dir, fname) 
                for fname in train_hammer_fnames[pic_index-10:pic_index]]

for i, img_path in enumerate(next_screwdriver_pix+next_wrench_pix+next_hammer_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
import tensorflow as tf

**The "output shape" column shows how the size of your feature map evolves in each successive layer. The convolution layers reduce the size of the feature maps by a bit due to padding, and each pooling layer halves the dimensions.**

# Pre-Trained Model

In [ ]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'


pre_trained_model = InceptionV3(input_shape = (300,300, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

# Data Preprocessing

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=30,
                                                    class_mode='categorical',
                                                    target_size=(300,300))     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=30,
                                                         class_mode  = 'categorical',
                                                         target_size = (300,300))

**We then add convolutional layers as in the previous example, and flatten the final result to feed into the densely connected layers.**
**Note that because we are facing a two-class classification problem, i.e. a binary classification problem, we will end our network with a sigmoid activation, so that the output of our network will be a single scalar between 0 and 1, encoding the probability that the current image is class 1 (as opposed to class 0).**

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(128, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense(3, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])


In [ ]:
model.summary()

In [ ]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=20,
                              epochs=40,
                              validation_steps=10,
                              verbose=2)

In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'  )

In [ ]:

import cv2
img = cv2.imread('../input/mechanical-tools-dataset/test_data/test_data/Screwdriver (1415).JPEG')
plt.imshow(img)
img = cv2.resize(img,(300, 300))
img = np.reshape(img,[1,300, 300,3])

classes = model.predict(img)
print(classes)

In [ ]:
img = cv2.imread('../input/mechanical-tools-dataset/test_data/test_data/Screwdriver (1401).JPEG')
plt.imshow(img)
img = cv2.resize(img,(300, 300))
img = np.reshape(img,[1,300, 300,3])

classes = model.predict(img)
print(classes)

In [ ]:
img = cv2.imread('../input/mechanical-tools-dataset/test_data/test_data/Screwdriver (1402).JPEG')
plt.imshow(img)
img = cv2.resize(img,(300, 300))
img = np.reshape(img,[1,300, 300,3])

classes = model.predict(img)
print(classes)

In [ ]:
img = cv2.imread('../input/mechanical-tools-dataset/test_data/test_data/Wrench (286).JPEG')
plt.imshow(img)
img = cv2.resize(img,(300, 300))
img = np.reshape(img,[1,300, 300,3])

classes = model.predict(img)
print(classes)

**Visualizing Intermediate Representations
To get a feel for what kind of features our convnet has learned, one fun thing to do is to visualize how an input gets transformed as it goes through the convnet.
Let's pick a random image from the training set, and then generate a figure where each row is the output of a layer, and each image in the row is a specific filter in that output feature map. Rerun this cell to generate intermediate representations for a variety of training images.**

In [ ]:
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]
#visualization_model = Model(img_input, successive_outputs)
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)
# Let's prepare a random input image from the training set.
wrench_img_files = [os.path.join(train_wrench_dir, f) for f in train_wrench_fnames]
screwdriver_img_files = [os.path.join(train_wrench_dir, f) for f in train_screwdriver_fnames]
img_path = random.choice(wrench_img_files + screwdriver_img_files)

img = load_img(img_path, target_size=(300,300))  # this is a PIL image
x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

# Rescale by 1/255
x /= 255

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers[1:]]

# Now let's display our representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
    if len(feature_map.shape) == 4:
    # Just do this for the conv / maxpool layers, not the fully-connected layers
        n_features = feature_map.shape[-1]  # number of features in feature map
    # The feature map has shape (1, size, size, n_features)
        size = feature_map.shape[1]
    # We will tile our images in this matrix
        display_grid = np.zeros((size, size * n_features))
        for i in range(n_features):
      # Postprocess the feature to make it visually palatable
            x = feature_map[0, :, :, i]
            x -= x.mean()
            x /= x.std()
            x *= 64
            x += 128
            x = np.clip(x, 0, 255).astype('uint8')
      # We'll tile each filter into this big horizontal grid
            display_grid[:, i * size : (i + 1) * size] = x
    # Display the grid
        scale = 20. / n_features
        plt.figure(figsize=(scale * n_features, scale))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='summer')